<a href="https://colab.research.google.com/github/tatianaportsova/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/Copy_of_LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [1]:
! wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2020-01-21 15:58:26--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.06s   

2020-01-21 15:58:26 (278 KB/s) - ‘house-votes-84.data’ saved [18171/18171]



In [49]:
import pandas as pd

column_headers = ['party', 'hadicapped-infants', 'water-project', 'budget',
                  'physician-fee-freeze', 'el-salvador-aid', 'religious-groups',
                  'anti-satellite-ban', 'aid-to-contras', 'mx-missile',
                  'immigration', 'synfuels', 'education', 'right-to-sue',
                  'crime', 'duty-free', 'south-africa']

df = pd.read_csv('house-votes-84.data', header = None, names = column_headers,
                 na_values = '?')

print(df.shape)
df.head()

(435, 17)


,party,hadicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [50]:
df = df.replace({'y':1, 'n':0})
df.head(3)

,party,hadicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0


In [51]:
df['party'].value_counts()

democrat      267
republican    168
Name: party, dtype: int64

In [52]:
dem = df[df['party']== 'democrat']
dem.head()

,party,hadicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


In [94]:
rep = df[df['party']== 'republican']
rep.head()

,party,hadicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


In [95]:
dem.describe()

,hadicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
count,258.000000,239.000000,260.000000,259.000000,255.000000,258.000000,259.000000,263.000000,248.000000,263.000000,255.000000,249.000000,252.000000,257.000000,251.000000,185.000000
mean,0.604651,0.502092,0.888462,0.054054,0.215686,0.476744,0.772201,0.828897,0.758065,0.471483,0.505882,0.144578,0.289683,0.350195,0.637450,0.935135
std,0.489876,0.501045,0.315405,0.226562,0.412106,0.500430,0.420224,0.377317,0.429121,0.500138,0.500949,0.352383,0.454518,0.477962,0.481697,0.246956
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [96]:
rep.describe()

,hadicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
count,165.000000,148.000000,164.000000,165.000000,165.000000,166.000000,162.000000,157.000000,165.000000,165.000000,159.000000,155.000000,158.000000,161.000000,156.000000,146.000000
mean,0.187879,0.506757,0.134146,0.987879,0.951515,0.897590,0.240741,0.152866,0.115152,0.557576,0.132075,0.870968,0.860759,0.981366,0.089744,0.657534
std,0.391804,0.501652,0.341853,0.109760,0.215442,0.304104,0.428859,0.361010,0.320176,0.498186,0.339643,0.336322,0.347298,0.135649,0.286735,0.476168
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000
75%,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [0]:
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel
from scipy import stats

## 2-sample T-test

1) Null Hypothesis: There is no difference between the average voting rates (levels of support for the _____ bill between the democrats and republicans in the house of representatives.

2) Alternative Hypothesis: Level of support between the two parties differs.

3) 95% Confidence Level.

## 1-sample T-test

1) Null Hypothesis: Average Dem or Rep support for ____ bill is 100%

2) Alternative Hypothesis: Average support isn't equal to 100%

# **Handicapped infants**
Based on a p-value of 1.61344 and a test statistic of 9.20526, I **reject** the null hypothesis that democrat support for the Handicapped infants project bill is equal to republican support for the bill, and I suggest the alternative that the support for this bill between the two parties is different.

In [75]:
ttest_ind(dem['hadicapped-infants'],rep['hadicapped-infants'], nan_policy='omit')

Ttest_indResult(statistic=9.205264294809222, pvalue=1.613440327937243e-18)

# **Water project**
Based on a p-value of 0.92915 and a test statistic of -0.08896, I **fail to reject** the null hypothesis that democrat support for the Water project bill is equal to republican support for the bill.

In [102]:
ttest_ind(dem['water-project'],rep['water-project'],nan_policy='omit')

Ttest_indResult(statistic=-0.08896538137868286, pvalue=0.9291556823993485)

# **Budget**
Based on a p-value of 2.0703402 and a test statistic of 23.2127769, I **reject** the null hypothesis that democrat support for the Budget bill is equal to republican support for the bill, and I suggest the alternative that the support for this bill between the two parties is different.

In [59]:
ttest_ind(dem['budget'],rep['budget'], nan_policy='omit')

Ttest_indResult(statistic=23.21277691701378, pvalue=2.0703402795404463e-77)

# **Physician fee-freeze**
Based on a p-value of 1.99426 and a test statistic of -49.36708, I **reject** the null hypothesis that democrat support for the Physician fee-freeze bill is equal to republican support for the bill, and I suggest the alternative that the support for this bill between the two parties is different.

In [60]:

ttest_ind(dem['physician-fee-freeze'],rep['physician-fee-freeze'], nan_policy='omit')

Ttest_indResult(statistic=-49.36708157301406, pvalue=1.994262314074344e-177)

An issue that republicans support more than democrats.

I **fail to reject** the null hypothesis that the majority of republicans support the bill.

In [101]:
stats.ttest_1samp(rep['physician-fee-freeze'], 1, nan_policy='omit')

Ttest_1sampResult(statistic=-1.4185450076223511, pvalue=0.1579292482594923)

# **El Salvador aid**
Based on a p-value of 5.60052 and a test statistic of -21.136692, I **reject** the null hypothesis that democrat support for the El Salvador aid bill is equal to republican support for the bill, and I suggest the alternative that the support for this bill between the two parties is different.

In [61]:
ttest_ind(dem['el-salvador-aid'],rep['el-salvador-aid'], nan_policy='omit')

Ttest_indResult(statistic=-21.13669261173219, pvalue=5.600520111729011e-68)

# **Religious groups**
Based on a p-value of 2.39367 and a test statistic of -9.73757, I **reject** the null hypothesis that democrat support for the Religious groups bill is equal to republican support for the bill, and I suggest the alternative that the support for this bill between the two parties is different.
                  

In [62]:
ttest_ind(dem['religious-groups'],rep['religious-groups'], nan_policy='omit')

Ttest_indResult(statistic=-9.737575825219457, pvalue=2.3936722520597287e-20)

# **Anti-satellite ban**
Based on a p-value of 8.52103 and a test statistic of 12.52618, I **reject** the null hypothesis that democrat support for the Anti-satellite ban bill is equal to republican support for the bill, and I suggest the alternative that the support for this bill between the two parties is different.

In [63]:
ttest_ind(dem['anti-satellite-ban'],rep['anti-satellite-ban'], nan_policy='omit')

Ttest_indResult(statistic=12.526187929077842, pvalue=8.521033017443867e-31)

# **Aid to contras**
Based on a p-value of 2.82471 and a test statistic of 18.05293, I **reject** the null hypothesis that democrat support for the Aid to contras bill is equal to republican support for the bill, and I suggest the alternative that the support for this bill between the two parties is different.

In [64]:
ttest_ind(dem['aid-to-contras'],rep['aid-to-contras'], nan_policy='omit')

Ttest_indResult(statistic=18.052093200819733, pvalue=2.82471841372357e-54)

# **Mx-missile**
Based on a p-value of 5.03079 and a test statistic of 16.437503, I **reject** the null hypothesis that democrat support for the Mx-missile bill is equal to republican support for the bill, and I suggest the alternative that the support for this bill between the two parties is different.

In [65]:
ttest_ind(dem['mx-missile'],rep['mx-missile'], nan_policy='omit')

Ttest_indResult(statistic=16.437503268542994, pvalue=5.03079265310811e-47)

# **Immigration**
Based on a p-value of 0.083302 and a test statistic of -1.73591, I **fail to reject** the null hypothesis that democrat support for the Immigration bill is equal to republican support for the bill.

In [103]:
ttest_ind(dem['immigration'],rep['immigration'], nan_policy='omit')

Ttest_indResult(statistic=-1.7359117329695164, pvalue=0.08330248490425066)

# **Synfuels**
Based on a p-value of 1.57593 and a test statistic of 8.29360, I **reject** the null hypothesis that democrat support for the Synfuels bill is equal to republican support for the bill, and I suggest the alternative that the support for this bill between the two parties is different.

In [67]:
ttest_ind(dem['synfuels'],rep['synfuels'], nan_policy='omit')

Ttest_indResult(statistic=8.293603989407588, pvalue=1.5759322301054064e-15)

# **Education**
Based on a p-value of 1.88342 and a test statistic of -20.50068, I **reject** the null hypothesis that democrat support for the Education bill is equal to republican support for the bill, and I suggest the alternative that the support for this bill between the two parties is different.

In [68]:
ttest_ind(dem['education'],rep['education'], nan_policy='omit')

Ttest_indResult(statistic=-20.500685724563073, pvalue=1.8834203990450192e-64)

# **Right to sue**
Based on a p-value of 1.22785 and a test statistic of -13.51064, I **reject** the null hypothesis that democrat support for the Right to sue bill is equal to republican support for the bill, and I suggest the alternative that the support for this bill between the two parties is different.

In [69]:
ttest_ind(dem['right-to-sue'],rep['right-to-sue'], nan_policy='omit')

Ttest_indResult(statistic=-13.51064251060933, pvalue=1.2278581709672758e-34)

# **Crime**
Based on a p-value of 9.95234 and a test statistic of -16.34208, I **reject** the null hypothesis that democrat support for the Crime bill is equal to republican support for the bill, and I suggest the alternative that the support for this bill between the two parties is different.

In [70]:
ttest_ind(dem['crime'],rep['crime'], nan_policy='omit')

Ttest_indResult(statistic=-16.342085656197696, pvalue=9.952342705606092e-47)

# **Duty-free**
Based on a p-value of 5.9976 and a test statistic of 12.8531, I **reject** the null hypothesis that democrat support for the Duty-free bill is equal to republican support for the bill, and I suggest the alternative that the support for this bill between the two parties is different.

In [71]:
ttest_ind(dem['duty-free'],rep['duty-free'], nan_policy='omit')

Ttest_indResult(statistic=12.853146132542978, pvalue=5.997697174347365e-32)

# **South Africa**
Based on a p-value of 3.6526 and a test statistic of 6.8494, I **reject** the null hypothesis that democrat support for the South Africa bill is equal to republican support for the bill, and I suggest the alternative that the support for this bill between the two parties is different.

In [72]:
ttest_ind(dem['south-africa'],rep['south-africa'], nan_policy='omit')

Ttest_indResult(statistic=6.849454815841208, pvalue=3.652674361672226e-11)

An issue that democrats support more than republicans.

I **reject** the null hypothesis that democrat support for the bill is 100%, I suggest that the party is split on the issue.

In [100]:
stats.ttest_1samp(dem['south-africa'], 1, nan_policy='omit')

Ttest_1sampResult(statistic=-3.572534923600723, pvalue=0.0004513789099729073)

_




**Codes** that don't give the result I'm looking for

In [0]:
import numpy as np

In [0]:
dem1 = dem.dropna()
rep1 = rep.dropna()

In [112]:
most_significant = (0,0,'')
least_significant = (0,0,'')
for col in df.columns:
  if col == 'party':
    continue
  test = ttest_ind(dem1[col], rep1[col])
  if test[0] > most_significant[0]:
    most_significant = (test[0], test[1], col)
  if test[0] < least_significant[0]:
    least_significant = (test[0], test[1], col)

most_significant, least_significant

((14.742943396634677, 4.458031105913572e-35, 'budget'),
 (-41.94745401614099, 9.726393032601933e-110, 'physician-fee-freeze'))

In [113]:
most_significant = (0,0,'')
least_significant = (0,0,'')
for col in df.columns:
  if col == 'party':
    continue
  test = ttest_rel(dem1[col][:108], rep1[col])
  if test[0] > most_significant[0]:
    most_significant = (test[0], test[1], col)
  if test[0] < least_significant[0]:
    least_significant = (test[0], test[1], col)

most_significant, least_significant

((14.098076699374221, 3.820579719040378e-26, 'aid-to-contras'),
 (-42.649736224272246, 5.512291609153565e-69, 'physician-fee-freeze'))

In [114]:
most_significant = (0,0,'')
least_significant = (0,0,'')
for col in df.columns:
  if col == 'party':
    continue
  test = ttest_ind_from_stats(np.mean(dem[col]), np.std(dem1[col]),len(dem1[col]), np.mean(rep1[col]), np.std(rep1[col]),len(rep1[col]))
  if test[0] > most_significant[0]:
    most_significant = (test[0], test[1], col)
  if test[0] < least_significant[0]:
    least_significant = (test[0], test[1], col)

most_significant, least_significant

((15.520893358932643, 1.1925632807781907e-37, 'budget'),
 (-41.8680759054674, 1.4300018956802694e-109, 'physician-fee-freeze'))

In [115]:
highest_p = (0,0,'')
for col in df.columns:
  if col == 'party':
    continue
  test = ttest_ind(dem1[col], rep1[col])
  if test[1] > highest_p[1]:
    highest_p = (test[0], test[1], col)
highest_p

(-0.3128208943045492, 0.7547001979801332, 'water-project')

In [116]:
highest_p = (0,0,'')
for col in df.columns:
  if col == 'party':
    continue
  test = ttest_rel(dem1[col][:108], rep1[col])
  if test[1] > highest_p[1]:
    highest_p = (test[0], test[1], col)
highest_p

(-0.29363376795889345, 0.7696065569087399, 'immigration')

In [117]:
highest_p = (0,0,'')
for col in df.columns:
  if col == 'party':
    continue
  test = ttest_ind_from_stats(np.mean(dem1[col]), np.std(dem1[col]),len(dem1[col]), np.mean(rep1[col]), np.std(rep1[col]),len(rep1[col]))
  if test[1] > highest_p[1]:
    highest_p = (test[0], test[1], col)
highest_p

(-0.31417833888655217, 0.753670380632758, 'water-project')

In [118]:
ttest_rel(rep1['budget'], dem1['budget'][:108])

Ttest_relResult(statistic=-13.693418446022312, pvalue=2.8656875973851773e-25)